In [115]:
import pickle
import numpy as np
import pandas as pd
import nltk
import string
import sys
import matplotlib.pyplot as plt

from random import randint, choices
from tqdm import tqdm
from itertools import permutations
from scipy.sparse import dok_matrix, find
from scipy.spatial.distance import cosine, euclidean, jaccard

In [2]:
def __get_words(sentence):
    """
    Given a sentence, parses it's tokens, removing punctuation, stopwords and small words;
    It yields each word, one at a time.
    """
    #stopwords = set(map(str.lower, nltk.corpus.stopwords.words("english")))
    punctuation = set(string.punctuation)
    for word in nltk.tokenize.wordpunct_tokenize(sentence):
        word = word.lower()
        if (word.isalnum()) \
        and (word not in punctuation):
            yield word 

            
def get_vocabulary(documents) -> dict:
    """
    Given a list of paragraphs, iterates over it's sentences. 
    Every time a new word is found, it is added to the dictionary of words with a unique integer reference.
    """
    all_words = {}
    #sentences = []
    i=0
    
    for doc in tqdm(documents):
        for sentence in nltk.sent_tokenize(doc):
            for word in __get_words(sentence):
                if word not in all_words:
                    all_words[ word ] = i
                    i+=1                           
    return all_words

In [3]:
df = pd.read_csv("data/arxiv_data.csv")

In [4]:
words = get_vocabulary(df.summaries)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51774/51774 [00:19<00:00, 2614.45it/s]


In [5]:
rev_words = {item[1]:item[0] for item in words.items()}

In [6]:
len(words)

58933

In [7]:
def get_doc(idx):
    return df.loc[idx, "summaries"]

In [8]:
def word2int(word):
    return words[word]

def int2word(idx):
    return rev_words[idx]

In [9]:
def get_most_similar_from_buckets(buckets, n_docs):
    distances = { i:{} for i in range(n_docs)}
    
    for key in tqdm(buckets):
        bucket = buckets[key]
        for pair in permutations(bucket, 2):
            distances_to_doc_a = distances[pair[0]]
            distances_to_doc_a[pair[1]] = distances_to_doc_a.get(pair[1], 0) + 1
            
    for key in tqdm(distances.keys()):
        docs = list(distances[key].keys())
        counts = list(distances[key].values())

        order = np.argsort(counts)[::-1]
        distances[key] = [docs[i] for i in order]
    return distances

In [67]:
def count_true_and_false_positive(candidates, similars):
    true_positive = 0
    false_positive = 0
    
    for doc_i in tqdm(candidates):
        true_positive += len(similars[doc_i])
        false_positive+= len(candidates[doc_i]) - len(similars[doc_i])
    return true_positive, false_positive

## LSHT for Jaccard Similarity | Bag of Words Representation

In [ ]:
def JaccardSim(d1, d2):    
    a =np.inner(d1,d2)
    bc=np.sum(d1+d2)-a
    return a/bc

In [ ]:
def filter_similar_from_buckets_jacard(potential_similar_docs, matrix, thresh=0.7):
    similar_docs_filtered = {}
    for doc_i, docs_j in tqdm(potential_similar_docs.items()):
        dists =JaccardSim(
                matrix[doc_i].toarray(),
                matrix[docs_j].toarray()
            ).ravel()
        
        
        candidates = potential_similar_docs[doc_i]
        similar_docs_filtered[doc_i] = candidates[dists>=thresh]
    return similar_docs_filtered

In [54]:
def to_bag_of_words(documents, vocabulary):
    N = len(documents)
    docs = {i:set() for i in range(N)}#dok_matrix((len(documents), len(vocabulary)))
    
    for i, d in tqdm(enumerate(documents)):
        for sentence in nltk.sent_tokenize(d):
            for word in __get_words(sentence):
                col = vocabulary[word]
                docs[i]|= {col}
    
    sparse_docs = dok_matrix((len(documents), len(vocabulary)))
    for row, cols in tqdm(docs.items()):
        cols = list(cols)
        sparse_docs[row, cols]=1
    return sparse_docs

In [175]:
docs = to_bag_of_words(df.summaries, words).tocsc()

51774it [00:22, 2267.14it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51774/51774 [00:10<00:00, 5176.09it/s]


In [56]:
docs.shape

(51774, 58933)

In [57]:
def get_buckets(documents, permutations, N, B, R, NB):
    buckets = {}
    
    docs_set = set(range(N))
    
    for band in tqdm(range(B)):
        signatures = np.zeros((N, R), dtype=int)
        for r in range(R):
            current_perm = permutations[band*R + r]
            L = docs_set.copy()
            i=0
            while len(L)>0:
                elem = current_perm[i]
                docs_found = documents[elem] & L
                
                if len(docs_found)>0:
                    signatures[list(docs_found), r] = i
                    L -= docs_found
                i+=1
                if i==N:
                    signatures[list(L),r]=i
                    L = {}
        
        for doc in range(N):
            bucket = hash(tuple(signatures[doc]))%NB
            buckets.setdefault((band, bucket), set()).add(doc)
    return buckets

In [58]:
def LSHT(documents, B, R, NB=28934501):
    N, M = documents.shape
    
    #d_transpose = documents.T
    d_transpose = []
    for i in tqdm(range(M)):
        d_transpose.append( 
            set( find( documents[:, i] )[0] )
        )
    
    P = B*R
    permutations = np.array([np.random.permutation(M) for _ in range(P)])
    buckets = get_buckets(d_transpose, permutations, N, B, R, NB)
    return buckets
    

### Estimate good choices of B and R for high positive rate and low positive rate

In [187]:
subdocs = docs[choices(range(docs.shape[0]), k=10000)]
buckets = LSHT(subdocs, 40, 10)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:10<00:00,  3.68it/s]


In [188]:
len(buckets)

344809

In [189]:
similar = get_most_similar_from_buckets(buckets, subdocs.shape[0])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 149284.74it/s]


In [190]:
similar = { doc_i:np.array(docs_j) for doc_i, docs_j in tqdm(similar.items())}

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 842229.72it/s]


In [191]:
similar[0]

array([], dtype=float64)

In [192]:
true_sim = filter_similar_from_buckets_jacard(similar, subdocs)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [01:37<00:00, 102.63it/s]


In [193]:
t,f = count_true_and_false_positive(similar, true_sim)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 1607752.22it/s]


In [194]:
t/(t+f)

0.6469512195121951

## LSHT for Cosine Similarity | TF-IDF Representation

In [161]:
def filter_similar_from_buckets(potential_similar_docs, matrix, thresh=0.7):
    similar_docs_filtered = {}
    for doc_i, docs_j in tqdm(potential_similar_docs.items()):
        dists = 1 - np.array([
            cosine(
                matrix[doc_i].toarray().ravel(),
                matrix[doc_j].toarray().ravel()
            ) for doc_j in docs_j
        ])
        
        candidates = potential_similar_docs[doc_i]
        similar_docs_filtered[doc_i] = candidates[dists>=thresh]
    return similar_docs_filtered

In [10]:
def process_raw_text(documents):
    new_docs = []
    for i, d in tqdm(enumerate(documents)):
        current = []
        for sentence in nltk.sent_tokenize(d):
            for word in __get_words(sentence):
                current.append( word)
        new_docs.append(current)
    return new_docs

In [11]:
def count_words(doc) -> dict[str, int]:
    """
    Counts the ocurrence of each word in the document corpus.
    """
    #return dict(zip(*np.unique(doc, return_counts=True)))
    return np.unique(doc, return_counts=True)
    
def get_tf_matrix(docs, vocab):
    N, M = len(docs), len(vocab)
    tf_matrix = dok_matrix((N, M))
    
    for i, doc in tqdm(enumerate(docs)):
        #calc document  tf vector
        words, counts = count_words(doc)
        if len(words)>0:
            max_value = counts.max()
            
            words_idx = list(map(word2int, words))
            tf_matrix[i, words_idx] = counts/max_value
    return tf_matrix

def get_idf_matrix(docs, vocab):
    N = len(docs)
    word_counts = np.zeros(len(vocab))
    
    for doc in tqdm(docs):
        for word in np.unique(doc):
            word_counts[ word2int(word) ] += 1
    
    return np.log2( (N + 1) / (word_counts + 1) )


def get_tf_idf(docs, vocab):
    tf = get_tf_matrix(docs, vocab).tocsr()
    idf = get_idf_matrix(docs, vocab)
    
    return tf.multiply(idf)

In [195]:
docs = process_raw_text(df.summaries)
tfidf = get_tf_idf(docs, words).tocsr()

51774it [00:21, 2442.88it/s]
51774it [00:23, 2234.49it/s]
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 51774/51774 [00:09<00:00, 5500.77it/s]


### Estimate good B and R parameters

In [196]:
subtf = tfidf[ choices(range(tfidf.shape[0]), k=10000)]

In [16]:
def get_buckets_cosine(documents, vectors, N, B, R, NB):
    buckets = {}
    signatures = np.where( (documents @ vectors) <= 0, 0, 1)
    binary_power = 2**np.arange(R)
    
    for band in tqdm(range(B)):        
        band_signatures = signatures[:, band*R:band*R+R]
        ##print(band_signatures)
        #print(band_signatures.shape)
        
        for doc in range(N):
            bucket = hash(tuple(band_signatures[doc]))%NB
            buckets.setdefault((band, bucket), set()).add(doc)
    return buckets

In [17]:
def LSHT_cosine(documents, B, R, NB=28934501):
    N, M = documents.shape
    
    P = B*R
    v_vectors = np.where(np.random.random(size=(M, P))<=0.5, -1, 1)
    buckets = get_buckets_cosine(documents, v_vectors, N, B, R, NB)
    return buckets

In [197]:
buckets = LSHT_cosine(subtf, 40, 30)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 40/40 [00:01<00:00, 26.91it/s]


In [198]:
sims = get_most_similar_from_buckets(buckets, subtf.shape[0])

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 151513.00it/s]


In [199]:
sims = { doc_i:np.array(docs_j) for doc_i, docs_j in tqdm(sims.items())}

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 909196.22it/s]


In [169]:
sims[0]

array([], dtype=float64)

In [200]:
true_sims = filter_similar_from_buckets(sims, subtf)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:03<00:00, 2520.21it/s]


In [201]:
t,f =count_true_and_false_positive(sims, true_sims)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10000/10000 [00:00<00:00, 1657434.60it/s]


In [202]:
t/(t+f)

0.9540722596448254